In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *
from utils.Backtest import Backtest

In [3]:
os.listdir('./data/5m')

['BNBBTC-5m-data.csv',
 'BTCUSDT-5m-data.csv',
 'ETHBTC-5m-data.csv',
 'LTCBTC-5m-data.csv',
 'RVNBTC-5m-data.csv']

In [2]:
WINDOW_SIZE = 24

In [10]:
# for price in os.listdir('./data/5m'):
for price in ['ETHBTC-5m-data.csv', 'LTCBTC-5m-data.csv', 'RVNBTC-5m-data.csv']:
    df = pd.read_csv(f'./data/5m/{price}')
    raw_data = DataLabeling(df, WINDOW_SIZE)
    train_df, val_df, test_df = train_val_test_split(raw_data.TI_data, rolling=80000)
    Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
    model = CDT_1D_model(WINDOW_SIZE, len(train_df.columns)-1)
    test_model = model.model
    test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path=f'5m_model/{price[:6]}_with_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)])

Epoch 1/20000
11867/11867 [==============================] - 151s 13ms/step - loss: 0.8034 - accuracy: 0.5631 - val_loss: 27.1151 - val_accuracy: 0.3750
Epoch 2/20000
11867/11867 [==============================] - 148s 12ms/step - loss: 0.7563 - accuracy: 0.5792 - val_loss: 12.2826 - val_accuracy: 0.3750
Epoch 3/20000
11867/11867 [==============================] - 151s 13ms/step - loss: 0.7429 - accuracy: 0.5888 - val_loss: 1.6376 - val_accuracy: 0.3750
Epoch 4/20000
11867/11867 [==============================] - 147s 12ms/step - loss: 0.7312 - accuracy: 0.5935 - val_loss: 1.6087 - val_accuracy: 0.3750
Epoch 5/20000
11867/11867 [==============================] - 147s 12ms/step - loss: 0.7260 - accuracy: 0.5950 - val_loss: 1.8378 - val_accuracy: 0.3750
Epoch 6/20000
11867/11867 [==============================] - 150s 13ms/step - loss: 0.7241 - accuracy: 0.5956 - val_loss: 2.7855 - val_accuracy: 0.3750
Epoch 7/20000
11867/11867 [==============================] - 157s 13ms/step - loss: 0.

In [9]:
def train_val_test_split_1h(data, train_size=17520, val_size=1440, test_size=336, interval=336, rolling=0):
    '''
    1h data, use 2 years as training data, 2 months as validation data, and 2 weeks as testing data
    '''
    data_length = rolling+train_size+2*interval+val_size+test_size
    if len(data.index) < data_length:
        print(f'At least reduce rolling by {data_length - len(data.index)}')
        return
    
    total_size = train_size+val_size+test_size+2*interval+rolling
    if total_size > len(data.iloc[rolling:]):
        print('Out of range, please set rolling smaller')
        exit()
    
    train_df = data.iloc[rolling:rolling+train_size]
    val_start = rolling+train_size+interval
    test_start = rolling+train_size+2*interval+val_size
    val_df = data.iloc[val_start:val_start+val_size]
    test_df = data.iloc[test_start:test_start+test_size]

    return train_df, val_df, test_df

In [9]:
# for price in os.listdir('./data/1h'):
for price in ['BNBBTC-1h-data.csv', 'BTCUSDT-1h-data.csv']:
    df = pd.read_csv(f'./data/1h/{price}')
    raw_data = DataLabeling(df, WINDOW_SIZE)
    train_df, val_df, test_df = train_val_test_split_1h(raw_data.labelled_data)
    Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
    model = MLP_model(WINDOW_SIZE, len(train_df.columns)-1)
    test_model = model.model
    test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path=f'1h_model/{price[:6]}_without_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)])

Epoch 1/20000
1459/1459 [==============================] - 31s 20ms/step - loss: 0.8749 - accuracy: 0.4916 - val_loss: 0.7407 - val_accuracy: 0.6765
Epoch 2/20000
1459/1459 [==============================] - 29s 20ms/step - loss: 0.7305 - accuracy: 0.5073 - val_loss: 0.7027 - val_accuracy: 0.6373
Epoch 3/20000
1459/1459 [==============================] - 28s 19ms/step - loss: 0.7199 - accuracy: 0.5102 - val_loss: 0.6879 - val_accuracy: 0.6275
Epoch 4/20000
1459/1459 [==============================] - 24s 17ms/step - loss: 0.7173 - accuracy: 0.5051 - val_loss: 0.6885 - val_accuracy: 0.6275
Epoch 5/20000
1459/1459 [==============================] - 24s 17ms/step - loss: 0.7153 - accuracy: 0.5087 - val_loss: 0.7060 - val_accuracy: 0.3725
Epoch 6/20000
1459/1459 [==============================] - 24s 17ms/step - loss: 0.7162 - accuracy: 0.5080 - val_loss: 0.7043 - val_accuracy: 0.4706
Epoch 7/20000
1459/1459 [==============================] - 24s 17ms/step - loss: 0.7150 - accuracy: 0.5041

In [16]:
for price in os.listdir('./data/5m'):
    df = pd.read_csv(f'./data/5m/{price}')
    raw_data = DataLabeling(df, WINDOW_SIZE)
    train_df, val_df, test_df = train_val_test_split(raw_data.TI_data, rolling=80000)
    Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
    test_model = tf.keras.models.load_model(f'5m_model/{price[:6]}_with_TIs/CDT-1D_model')
    test_model.evaluate(Data.test)

7/7 [==============================] - 0s 6ms/step - loss: 0.9214 - accuracy: 0.6164


In [18]:
for price in os.listdir('./data/1h'):
    df = pd.read_csv(f'./data/1h/{price}')
    raw_data = DataLabeling(df, WINDOW_SIZE)
    train_df, val_df, test_df = train_val_test_split_1h(raw_data.TI_data)
    Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
    test_model = tf.keras.models.load_model(f'1h_model/{price[:6]}_with_TIs/CDT-1D_model')
    test_model.evaluate(Data.test)

27/27 [==============================] - 0s 6ms/step - loss: 2.7797 - accuracy: 0.5240
